In [1]:
import requests
import pandas as pd

pns_data = pd.read_csv(r'D:\Dev\2026\BigData\national_pension.csv')
pns_data = pns_data.set_index(['Age_Group', 'Year'])

pns_data

,,Subscriber_Count
Age_Group,Year,
18~19,2024,51565
20-24,2024,626593
25-29,2024,1903933
30-34,2024,2345031
35-39,2024,2172632
...,...,...
40-44,2012,2497327
45-49,2012,2291974
50-54,2012,2394338


In [ ]:
# 2012년